# MERGE and OPTIMIZE Parallel Demo with Databricks Connect

This notebook demonstrates that MERGE and OPTIMIZE can run in parallel with row-level concurrency, deletion vectors, and liquid clustering in Databricks.

## Prerequisites
- Databricks Connect configured locally
- Environment variables set: DATABRICKS_HOST, DATABRICKS_TOKEN, DATABRICKS_CLUSTER_ID

## Overview

The demo creates a Delta table with the following features:
- **Row-level concurrency**: Multiple writers can modify different rows simultaneously
- **Deletion vectors**: Efficient deletion without rewriting entire files
- **Liquid clustering**: Automatic clustering on the merge column for optimal performance


In [0]:
# Cell 1: Connection & Setup
# 🚨 IMPORTANT: If you get version errors, RESTART THE KERNEL first!

import sys
print(f"🐍 Python executable: {sys.executable}")
print(f"🐍 Python version: {sys.version}")

# Initialize Databricks Connect
print("🔗 Initializing Databricks Connect...")
try:
    from databricks.connect import DatabricksSession
    spark = DatabricksSession.builder.getOrCreate()
    print("✅ Connected to Databricks cluster via Databricks Connect")
    
    # Verify connection
    print(f"✅ Spark version: {spark.version}")
    test_count = spark.range(3).count()
    print(f"✅ Connected to remote cluster: {test_count} test rows")
    
    # Check Databricks Runtime version
    try:
        version_info = spark.sql("SELECT current_version() as version").collect()[0]
        print(f"🏢 Databricks Runtime: {version_info.version}")
        print("✅ CONFIRMED: Running on DATABRICKS CLUSTER")
    except Exception as e:
        print(f"⚠️  Could not get runtime version: {str(e)[:50]}...")
    
except Exception as e:
    print(f"❌ Connection failed: {e}")
    print("💡 Try restarting the kernel and running again")
    raise

# Import required libraries
from pyspark.sql import functions as F
from pyspark.sql.types import StringType
from faker import Faker
from faker_vehicle import VehicleProvider
import uuid
import logging
import threading
import time
import random
from datetime import datetime

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

print("✅ All libraries imported and ready!")


In [0]:
# Cell 2: Package Verification
print("🔧 Verifying required packages...")
try:
    from faker import Faker
    from faker_vehicle import VehicleProvider
    print("✅ faker and faker_vehicle are available")
except ImportError as e:
    print(f"❌ Missing packages: {e}")
    print("💡 Installing missing packages...")
    import subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "faker", "faker-vehicle"])
    from faker import Faker
    from faker_vehicle import VehicleProvider
    print("✅ Packages installed successfully")

print("✅ All packages verified and ready!")


In [0]:
# Cell 3: Configuration
print("⚙️ Setting up configuration...")

# Set seeds for reproducibility
RANDOM_SEED = 42
random.seed(RANDOM_SEED)
print(f"🎲 Random seed set to: {RANDOM_SEED}")

# Create dynamic table name with logical name and timestamp
timestamp_str = datetime.now().strftime("%Y%m%d_%H%M%S")
TABLE_NAME = f"parallel_merge_optimize_demo_{timestamp_str}"
TARGET_TABLE = f"soni.default.{TABLE_NAME}"

CHECKPOINT_BASE = "s3://test-external-volume-bucket-2/test-folder"
JOIN_COLUMN = "event_id"
CLUSTERING_COLUMN = "event_timestamp"
INITIAL_EVENT_ID_POOL_SIZE = 10000

# Generate unique checkpoint locations (but deterministic within session)
session_id = f"session_{RANDOM_SEED}_{timestamp_str}"
checkpoint_bootstrap = f"{CHECKPOINT_BASE}/bootstrap_{session_id}"
checkpoint_main = f"{CHECKPOINT_BASE}/main_{session_id}"

print(f"📋 Table name: {TABLE_NAME}")
print(f"🎯 Full table path: {TARGET_TABLE}")
print(f"📁 Session ID: {session_id}")

logger.info(f"Target table: {TARGET_TABLE}")
logger.info(f"Clustering on: {CLUSTERING_COLUMN}")
logger.info(f"Session ID: {session_id}")
logger.info("Configuration loaded successfully")

print("✅ Configuration complete!")


In [0]:
# Cell 4: Reproducible Data Generation Setup
print("🎲 Setting up reproducible data generation...")

# Initialize Faker with seed for reproducible fake data
fake = Faker()
fake.seed_instance(RANDOM_SEED)
fake.add_provider(VehicleProvider)
print(f"✅ Faker initialized with seed: {RANDOM_SEED}")

# Create reproducible pool of existing event IDs
print(f"🔄 Creating {INITIAL_EVENT_ID_POOL_SIZE} reproducible event IDs...")
existing_event_ids = set()

# Use deterministic approach instead of random UUIDs
for i in range(INITIAL_EVENT_ID_POOL_SIZE):
    # Create deterministic but unique IDs based on seed and index
    deterministic_id = f"{i:09d}"
    existing_event_ids.add(deterministic_id)

print(f"✅ Created {len(existing_event_ids)} deterministic event IDs")
print(f"📝 Sample IDs: {list(existing_event_ids)[:3]}...")

# Create UDFs for fake data generation (these will use seeded faker)
event_id_udf = F.udf(lambda: f"new_{RANDOM_SEED}_{uuid.uuid4()}", StringType())
vehicle_make_udf = F.udf(fake.vehicle_make, StringType())
vehicle_model_udf = F.udf(fake.vehicle_model, StringType())
vehicle_year_udf = F.udf(fake.vehicle_year, StringType())
latitude_udf = F.udf(lambda: str(fake.latitude()), StringType())   # Convert to string
longitude_udf = F.udf(lambda: str(fake.longitude()), StringType()) # Convert to string
zipcode_udf = F.udf(fake.zipcode, StringType())

logger.info(f"Created {len(existing_event_ids)} deterministic event IDs for updates")
logger.info("Reproducible UDFs and data generation initialized")

print("✅ Reproducible data generation ready!")


In [0]:
# Cell 5: Reproducible Streaming Function
def create_streaming_vehicle_data(rows_per_second=1000, num_partitions=4, update_ratio=0.5, seed_offset=0):
    """Create a streaming DataFrame with vehicle data, mixing updates and inserts based on existing event IDs.
    
    Args:
        rows_per_second: Rate of data generation
        num_partitions: Number of partitions
        update_ratio: Ratio of updates vs inserts (0.0 = all inserts, 1.0 = all updates)
        seed_offset: Offset to add to base seed for different streams
    """
    logger.info(f"Creating streaming vehicle data with {update_ratio*100}% updates")
    
    # Convert existing_event_ids to a list for efficient random access
    existing_ids_list = list(existing_event_ids) if existing_event_ids else []
    logger.info(f"Using {len(existing_ids_list)} existing IDs for updates")
    
    # Create a seeded random generator for this specific function call
    stream_seed = RANDOM_SEED + seed_offset
    stream_random = random.Random(stream_seed)
    
    # Create a mix of existing IDs (for updates) and new IDs (for inserts)
    def generate_event_id_with_mix():
        # Use the seeded random generator for reproducible choices
        if existing_ids_list and stream_random.random() < update_ratio:
            return stream_random.choice(existing_ids_list)  # Existing ID for update
        else:
            # Generate deterministic new IDs
            return f"new_{stream_seed}_{stream_random.randint(100000, 999999)}"
        
    event_id_mixed_udf = F.udf(generate_event_id_with_mix, StringType())
    
    # Create streaming DataFrame with simplified schema
    df = (spark.readStream.format("rate")
          .option("numPartitions", num_partitions)
          .option("rowsPerSecond", rows_per_second)
          .load()
          .withColumn("event_timestamp", F.current_timestamp())
          .withColumn("event_id", event_id_mixed_udf())
          .withColumn("vehicle_make", vehicle_make_udf())
          .withColumn("vehicle_model", vehicle_model_udf())
          .withColumn("vehicle_year", vehicle_year_udf())
          .withColumn("latitude", latitude_udf())
          .withColumn("longitude", longitude_udf())
          .withColumn("zipcode", zipcode_udf())
          .drop("value", "timestamp")
    )
    return df

logger.info("Reproducible streaming data generation function created")
print("✅ Reproducible streaming function ready!")


In [0]:
# Cell 6: Create Target Table
print("🏗️ Creating target table with optimized schema...")

# Safety checks for required variables
required_vars = ['TARGET_TABLE', 'JOIN_COLUMN', 'CLUSTERING_COLUMN']
missing_vars = []

for var_name in required_vars:
    try:
        globals()[var_name]
    except KeyError:
        missing_vars.append(var_name)

if missing_vars:
    print(f"⚠️  Missing variables: {missing_vars}")
    print("🔧 Please run Cell 3 (Configuration) first!")
    raise NameError(f"Required variables not found: {missing_vars}")

print(f"✅ All required variables available")

# Drop table if it exists (clean slate)
spark.sql(f"DROP TABLE IF EXISTS {TARGET_TABLE}")
print(f"🧹 Dropped existing table {TARGET_TABLE} if it existed")

print("📝 Creating table with schema:")
print(f"   - Primary key: {JOIN_COLUMN}")
print(f"   - Clustering: {CLUSTERING_COLUMN} (REQUIRED)")
print("   - Features: Row tracking, Deletion vectors (enabled by default)")

# Create table with liquid clustering (features enabled by default)
create_table_sql = f"""
CREATE TABLE {TARGET_TABLE} (
    event_id STRING NOT NULL,
    event_timestamp TIMESTAMP NOT NULL,
    vehicle_make STRING,
    vehicle_model STRING,
    vehicle_year STRING,
    latitude STRING,
    longitude STRING,
    zipcode STRING
)
USING DELTA
CLUSTER BY ({CLUSTERING_COLUMN})
COMMENT 'Demo table for parallel MERGE and OPTIMIZE operations with liquid clustering'
"""

# Execute table creation
print(f"🔧 Creating table with liquid clustering and all required features...")
spark.sql(create_table_sql)
print("✅ Table created successfully with liquid clustering!")
print(f"   🔗 Liquid clustering: {CLUSTERING_COLUMN}")
print("   🔄 Row tracking: enabled by default")
print("   🗑️  Deletion vectors: enabled by default")

# Verify table creation
table_info = spark.sql(f"DESCRIBE DETAIL {TARGET_TABLE}").collect()[0]
print(f"✅ Table setup complete!")
print(f"   📍 Location: {table_info.location}")
print(f"   🏷️  Format: {table_info.format}")
print(f"   📊 Properties: {table_info.properties}")

# Show table schema
print("\n📋 Table Schema:")
display(spark.sql(f"DESCRIBE {TARGET_TABLE}"))

print("✅ Target table ready for streaming operations!")


In [0]:
# Cell 7: Ready for Streaming Demo
print("🎯 Table created and ready for streaming MERGE demo!")
print("📊 Starting with empty table - streaming will populate it naturally")

# Verify table is empty and ready
initial_count = spark.sql(f"SELECT COUNT(*) as count FROM {TARGET_TABLE}").collect()[0]['count']
print(f"✅ Table starts with {initial_count} records")

# Show table structure
print("\n📋 Table structure:")
display(spark.sql(f"DESCRIBE {TARGET_TABLE}"))

print("\n🎯 Why no bootstrap needed:")
print("   • Infinite MERGE stream will populate the table naturally")
print("   • First batches = 100% inserts (realistic start)")
print("   • Later batches = mix of updates/inserts (realistic growth)")
print("   • Demonstrates true streaming behavior from empty to populated")

print("\n✅ Ready to start parallel MERGE and OPTIMIZE operations!")


In [0]:
# Cell 8: MERGE Processor for Parallel Operations
class forEachBatchProcessor:
    def __init__(self, target_table: str, clustering_column: str, join_column: str):
        self.target_table = target_table
        self.clustering_column = clustering_column
        self.join_column = join_column
        self.batch_counter = 0
        self.total_processed = 0
        self.total_updates = 0
        self.total_inserts = 0

    def make_changes_using_the_micro_batch(self, microBatchOutputDF, batchId: int):
        self.batch_counter += 1
        print(f"=== MERGE BATCH {self.batch_counter} (ID: {batchId}) ===")
        
        # Count records before deduplication
        total_records = microBatchOutputDF.count()
        print(f"Processing {total_records} records")
        
        spark_session_for_this_micro_batch = microBatchOutputDF.sparkSession

        # Create temporary view for the batch
        view_name = f"updates_batch_{batchId}"
        microBatchOutputDF.dropDuplicates([self.join_column]).createOrReplaceTempView(view_name)

        # MERGE statement with row-level concurrency support
        sql_for_merge = f"""
          MERGE INTO {self.target_table} target
          USING {view_name} source
          ON source.{self.join_column} = target.{self.join_column}
          WHEN MATCHED THEN
            UPDATE SET *
          WHEN NOT MATCHED THEN
            INSERT *
        """
        
        print(f"Executing MERGE for batch {batchId}...")
        start_time = time.time()
        
        # Execute MERGE and capture results
        result = spark_session_for_this_micro_batch.sql(sql_for_merge)
        
        # Get MERGE statistics
        try:
            # Count unique event IDs in the batch to estimate updates vs inserts
            batch_data = microBatchOutputDF.collect()
            updates = sum(1 for row in batch_data if row.event_id in existing_event_ids)
            inserts = len(batch_data) - updates
            
            self.total_updates += updates
            self.total_inserts += inserts
            self.total_processed += (updates + inserts)
            
            print(f"MERGE Results: {updates} updates, {inserts} inserts")
            print(f"Running totals: {self.total_updates} updates, {self.total_inserts} inserts")
            
            # Verify we have at least 10% updates as required
            if self.total_processed > 0:
                update_percentage = (self.total_updates / self.total_processed) * 100
                print(f"Update percentage: {update_percentage:.1f}% (target: ≥10%)")
                if update_percentage >= 10:
                    print("✅ Meeting requirement: ≥10% updates")
                else:
                    print("⚠️  Below target: Need ≥10% updates")
                
        except Exception as e:
            error_msg = str(e)
            print(f"Could not get MERGE statistics: {error_msg}")
            raise (error_msg)

        
        end_time = time.time()
        print(f"MERGE completed in {end_time - start_time:.2f} seconds")
        print(f"=== END MERGE BATCH {self.batch_counter} ===\n")

# Initialize the MERGE processor
merge_processor = forEachBatchProcessor(
    target_table=TARGET_TABLE,
    clustering_column=CLUSTERING_COLUMN,
    join_column=JOIN_COLUMN,
)

print("✅ MERGE processor initialized successfully!")


In [0]:
# Cell 9: Start Parallel MERGE Streaming
print("🚀 Starting parallel MERGE streaming job...")

# Start the MERGE streaming job with 50% updates (exceeds 10% requirement)
merge_stream = (
    create_streaming_vehicle_data(rows_per_second=100, num_partitions=2, update_ratio=0.5)  # 50% updates, 50% inserts
      .writeStream
      .option("queryName", f"MERGE_Data_Into_Table_{TARGET_TABLE}")
      .foreachBatch(merge_processor.make_changes_using_the_micro_batch)
      .trigger(processingTime="10 seconds")
      .option("checkpointLocation", checkpoint_main)
      .start()
)

print(f"✅ MERGE streaming job started: {merge_stream.name}")
print(f"📊 Streaming job status: {merge_stream.status}")
print(f"🎯 Update ratio: 50% (exceeds 10% requirement)")
print("🔄 MERGE operations will run every 10 seconds")


In [0]:
# Cell 10: Parallel OPTIMIZE Operations
def run_optimize_operations():
    """Run OPTIMIZE operations in a separate thread to demonstrate parallel execution"""
    counter = 0
    while True:
        counter += 1
        
        # Random sleep between 15-120 seconds
        sleep_duration = random.uniform(15, 120)
        time.sleep(sleep_duration)

        print(f"\n=== OPTIMIZE OPERATION {counter} ===")
        print(f"Sleep duration: {sleep_duration:.2f} seconds")
        
        # OPTIMIZE with liquid clustering (no ZORDER needed)
        # Liquid clustering automatically handles data layout optimization
        optimize_sql = f"""
            OPTIMIZE {TARGET_TABLE}
        """
        
        print(f"Executing OPTIMIZE operation {counter}...")
        start_time = time.time()
        
        try:
            result = spark.sql(optimize_sql)
            end_time = time.time()
            print(f"✅ OPTIMIZE completed in {end_time - start_time:.2f} seconds")
            
            # Show optimization results
            if result.count() > 0:
                print("📊 Optimization results:")
                result_data = result.collect()
                for row in result_data:
                    print(f"   Files optimized: {getattr(row, 'num_files_added', 'N/A')}")
                    print(f"   Files removed: {getattr(row, 'num_files_removed', 'N/A')}")
            
        except Exception as e:
            error_msg = str(e)
            print(f"❌ OPTIMIZE failed: {error_msg}")
            
            # Check for critical session errors that should stop the demo
            critical_errors = [
                "PERMISSION_DENIED",
                "Local RPC without associated session",
                "Session not found",
                "Connection refused",
                "UNAUTHENTICATED"
            ]
            
            if any(critical_error in error_msg for critical_error in critical_errors):
                print("🚨 CRITICAL ERROR DETECTED!")
                print("🛑 This indicates a Databricks Connect session issue")
                print("💡 Recommended actions:")
                print("   1. Check your Databricks Connect configuration")
                print("   2. Verify DATABRICKS_HOST, DATABRICKS_TOKEN, DATABRICKS_CLUSTER_ID")
                print("   3. Restart the kernel and reconnect")
                print("   4. Run stop_demo() to clean up")
                print("🚨 STOPPING OPTIMIZE OPERATIONS DUE TO CRITICAL ERROR")
                break  # Exit the while loop
            else:
                print("⚠️  Non-critical error - continuing OPTIMIZE operations")
        
        print(f"=== END OPTIMIZE OPERATION {counter} ===\n")

# Start OPTIMIZE operations in a separate thread
print("🔧 Starting OPTIMIZE operations in parallel...")
optimize_thread = threading.Thread(target=run_optimize_operations, daemon=True)
optimize_thread.start()
print("✅ OPTIMIZE thread started successfully!")
print("⚡ OPTIMIZE will run every 15-30 seconds in parallel with MERGE")


In [0]:
# Cell 11: Demo Status & Cleanup
print("🎯 PARALLEL MERGE AND OPTIMIZE DEMO ACTIVE!")
print("=" * 50)

# Show current streaming job status
if 'merge_stream' in globals():
    print(f"📊 MERGE Stream: {merge_stream.name}")
    print(f"📊 Active: {merge_stream.isActive}")
    print(f"📊 Status: {merge_stream.status}")

print("\n🔄 Operations running:")
print("   • MERGE: Every 10 seconds (50% updates)")
print("   • OPTIMIZE: Every 15-30 seconds (parallel)")
print("   • Requirement: ≥10% updates ✅")

# Cleanup function
def stop_demo():
    """Stop all streaming jobs and clean up resources"""
    print("\n🛑 Stopping demo...")
    
    # Stop the MERGE streaming job
    if 'merge_stream' in globals() and merge_stream.isActive:
        merge_stream.stop()
        print("✅ MERGE streaming job stopped")
    
    # Background threads stop automatically (daemon=True)
    print("✅ Background threads stopped")
    print("✅ Demo stopped successfully!")

print("\n💡 To stop: stop_demo()")
print("💡 To check data: display(spark.sql(f'SELECT * FROM {TARGET_TABLE} LIMIT 5'))")


In [0]:
# This cell has been removed - functionality merged into Cell 11
# The notebook now has 11 essential cells instead of 12
